In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs
from tqdm.notebook import tqdm
import pyinterp.backends.xarray

xr.set_options(display_style='html')

from dask_gateway import Gateway
from dask.distributed import Client

gateway = Gateway()
cluster = gateway.new_cluster()
cluster.adapt(minimum=1, maximum=20)
client = Client(cluster)
cluster

In [74]:
def load_data(df):
    """
    Load data for given sourc
    """
    ds = {}
    for source_id in tqdm(df['source_id']):
        vad = df[(df.source_id == source_id)].zstore.values[0]
    
        gcs = gcsfs.GCSFileSystem(token='anon')
        if any(df.variable_id == 'tas'):
            ds[source_id] = xr.open_zarr(gcs.get_mapper(vad), consolidated=True).tas
        elif any(df.variable_id.str.contains('tasmin')):
            ds[source_id] = xr.open_zarr(gcs.get_mapper(vad), consolidated=True).tasmin
        elif any(df.variable_id.str.contains('tasmax')):
            ds[source_id] = xr.open_zarr(gcs.get_mapper(vad), consolidated=True).tasmax
        else:
            ds[source_id] = xr.open_zarr(gcs.get_mapper(vad), consolidated=True).pr
        
#         """
#         Interpolate to 1°×1° grid using inverse-distance weighting interpolation
#         """
#         interpolator = pyinterp.backends.xarray.2dGrid(
#             ds[source_id], geodetic=True)
#         mss = interpolator.inverse_distance_weighting(dict(lon=mx.flatten(), lat=my.flatten()))
    return ds

In [75]:
# Now we need to limit the spatial domain
# PNW domain: 124.5°W–110.5°W, 41.5°–49.5°N
# Expanded domain: 165°W–100°W, 20°N–60°N

def meanannualmean(df,y):
    ds = {}
    for source_id in tqdm(df.keys()):
        try:
            ds[source_id] = df[source_id].sel(time=slice(y[0],y[1]),lat=slice(41.5,49.5),lon=slice(235.5,249.5)).mean() 
        except ValueError:
            ds[source_id] = df[source_id].sel(time=slice(y[0],y[1]),latitude=slice(41.5,49.5),longitude=slice(235.5,249.5)).mean()
        except:
            print(source_id,'failed to compute annual mean')
    return ds

In [76]:
def meandiurnalrange(max,min,y):
    ds = {}
    for source_id in tqdm(max.keys()):
        try:
            tmx = max[source_id].sel(time=slice(y[0],y[1]),lat=slice(41.5,49.5),lon=slice(235.5,249.5)).groupby('time.season').mean(['time','lat','lon'])
            tmn = min[source_id].sel(time=slice(y[0],y[1]),lat=slice(41.5,49.5),lon=slice(235.5,249.5)).groupby('time.season').mean(['time','lat','lon'])
            ds[source_id] = tmx-tmn
        except:
            print(source_id,'failed to compute mean diurnal range')
#         print(source_id, ds[source_id].values)
    return ds

In [77]:
# Mean amplitude of seasonal cycle as the difference between warmest and coldest month (T) or wettest and driest month (P). 
# Monthly precipitation calculated as percentage of mean annual total, 1960–1999 or wettest and driest month (P). 
# Monthly precipitation calculated as percentage of mean annual total, 1960–1999.
    
def seasonamp(df,y):
    ds = {}
    for source_id in tqdm(df.keys()):
        try:
            var = df[source_id].sel(time=slice(y[0],y[1]),lat=slice(41.5,49.5),lon=slice(235.5,249.5)).mean(['lat','lon'])
            mx = var.groupby('time.year').max()
            mn = var.groupby('time.year').min()
            if df[source_id].units == 'K':
                ds[source_id] = mx-mn
            else:
                tot = var.groupby('time.year').sum()
                ds[source_id] = ((mx-mn)/tot)*100.
            ds[source_id] = ds[source_id].mean()
        except:
            print(source_id, 'failed to compute seasonal amplitude')
    return ds

In [ ]:
def main():
    
#==================================================
    # Gather data
    df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
    
    # Query all historical runs for needed variables
    tas = df.query("activity_id=='CMIP' & experiment_id=='historical' & member_id=='r1i1p1f1' & table_id=='Amon' & variable_id=='tas' & grid_label=='gn'")
    tasmin = df.query("activity_id=='CMIP' & experiment_id=='historical' & member_id=='r1i1p1f1' & table_id=='Amon' & variable_id=='tasmin' & grid_label=='gn'")
    tasmax = df.query("activity_id=='CMIP' & experiment_id=='historical' & member_id=='r1i1p1f1' & table_id=='Amon' & variable_id=='tasmax' & grid_label=='gn'")
    pr = df.query("activity_id=='CMIP' & experiment_id=='historical' & member_id=='r1i1p1f1' & table_id=='Amon' & variable_id=='pr' & grid_label=='gn'")
#==================================================
    print('Loading All Data...')
    
    tas = load_data(tas)
    tasmax = load_data(tasmax)
    tasmin = load_data(tasmin)
    pr = load_data(pr)
#==================================================
    print('Computing mean annual temperature and precipitation...')
    
    annmean_tas = meanannualmean(tas,['1960-01','1999-12'])
    annmean_pr = meanannualmean(pr,['1960-01','1999-12'])
#==================================================
    print('Computing mean diurnal temperature ranges for winter and summer seasons...')
    
    diurnal_tas = meandiurnalrange(tasmax,tasmin,['1950-01','1999-12']) # Something seems to be up with CMCC-CM2-SR5, AWI-CM-1-1-MR, and NESM3
#==================================================
    print('Computing mean seasonal amplitude of temperature and precipitation...')

    seasonamp_tas = seasonamp(tas,['1960-01','1999-12'])
    seasonamp_pr = seasonamp(pr,['1960-01','1999-12'])

In [79]:
if __name__ == "__main__":
    main()

/srv/conda/envs/notebook/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Loading All Data...



Computing mean annual temperature and precipitation...


MPI-ESM1-2-HR failed to compute annual mean




Computing mean diurnal temperature ranges for winter and summer seasons...



Computing mean seasonal amplitude of temperature and precipitation...


MPI-ESM1-2-HR failed to compute seasonal amplitude
MCM-UA-1-0 failed to compute seasonal amplitude



MCM-UA-1-0 failed to compute seasonal amplitude

